# Paths to our dataset files

In [1]:
data = '../datasets/bts_site_b_train/train.zip'
mapper = '../datasets/bts_site_b_train/mapper_TrainOnly.csv'
model = '../datasets/bts_site_b_train/Site_B.ttl'
schema = '../datasets/bts_site_b_train/Brick_v1.2.1.ttl'

# Load the Dataset

Hack for demo purposes to add the `src` directory to the path:

In [2]:
import sys
sys.path.append("../src")

Import and instantiate the manager:

In [3]:
from dbmgr import DBManager
db = DBManager(data, mapper, model, schema)

This `db` object is what is passed to each analytics module.

## Number of streams in the dataset

In [4]:
len(db)

668

## Iterating over the streams in the dataset

In [5]:
i = 0
for stream_id in db:
    print(stream_id)
    i += 1
    if i == 5:
        break

6f502ba0_77fb_43db_b6dc_48e68d5c822b
3c9ca09b_8952_45da_a066_bcb645cbbc68
d60c44c3_62fc_409a_a751_8b3ddeb9f197
eb0d5916_d942_4e2d_8129_60b83d984fdd
78078042_e16e_425b_9ec8_605aa149274e


## Check if a stream ID is in the dataset:

Not in the dataset:

In [6]:
'tim' in db

False

In the dataset:

In [7]:
'835a86d6_c235_4caf_8071_31b9bf24960b' in db

True

## Get a stream by its ID:

By indexing into the manager:

In [8]:
db['835a86d6_c235_4caf_8071_31b9bf24960b'].head()

,time,value,brick_class
0,2021-01-01 00:03:09.662,0.0,Enable_Status
1,2021-01-01 00:13:39.017,0.0,Enable_Status
2,2021-01-01 00:23:10.073,0.0,Enable_Status
3,2021-01-01 00:33:09.637,0.0,Enable_Status
4,2021-01-01 00:43:09.639,0.0,Enable_Status


By calling the getter:

In [9]:
db.get_stream('835a86d6_c235_4caf_8071_31b9bf24960b').head()

,time,value,brick_class
0,2021-01-01 00:03:09.662,0.0,Enable_Status
1,2021-01-01 00:13:39.017,0.0,Enable_Status
2,2021-01-01 00:23:10.073,0.0,Enable_Status
3,2021-01-01 00:33:09.637,0.0,Enable_Status
4,2021-01-01 00:43:09.639,0.0,Enable_Status


## Set a stream by its ID:

In [10]:
import pandas as pd

data = pd.DataFrame({
            "time": [pd.Timestamp('2017-03-22 15:16:45'), pd.Timestamp('2017-03-22 15:17:45'), pd.Timestamp('2017-03-22 15:18:45'), pd.Timestamp('2017-03-22 15:19:45') ],
            "value": [22.0, 23.0, 24.0, 25.0],
            "brick_class": ["Fake_Class", "Fake_Class", "Fake_Class", "Fake_Class"],
        })

By indexing into the manager:

In [11]:
db['835a86d6_c235_4caf_8071_31b9bf24960b'] = data
db['835a86d6_c235_4caf_8071_31b9bf24960b'].head()

,time,value,brick_class
0,2017-03-22 15:16:45,22.0,Fake_Class
1,2017-03-22 15:17:45,23.0,Fake_Class
2,2017-03-22 15:18:45,24.0,Fake_Class
3,2017-03-22 15:19:45,25.0,Fake_Class


By calling the setter:

In [12]:
db.set_stream('22543d51_3c38_46d3_98fe_a77838976df9', data)
db.get_stream('22543d51_3c38_46d3_98fe_a77838976df9').head()

,time,value,brick_class
0,2017-03-22 15:16:45,22.0,Fake_Class
1,2017-03-22 15:17:45,23.0,Fake_Class
2,2017-03-22 15:18:45,24.0,Fake_Class
3,2017-03-22 15:19:45,25.0,Fake_Class


## Get multiple streams by their IDs:

In [13]:
stream_ids = [
    '22543d51_3c38_46d3_98fe_a77838976df9',
    '3010a24a_2deb_43c8_9db4_cfc94c5faa5c',
    '405896a1_c195_422d_a0aa_18d2eca6a61d',
]
streams = db.get_streams(stream_ids)
for stream_id, stream in streams.items():
    print(f'========= {stream_id} =========')
    print(stream.head())
    print()

========= 22543d51_3c38_46d3_98fe_a77838976df9 =========
                 time  value brick_class
0 2017-03-22 15:16:45   22.0  Fake_Class
1 2017-03-22 15:17:45   23.0  Fake_Class
2 2017-03-22 15:18:45   24.0  Fake_Class
3 2017-03-22 15:19:45   25.0  Fake_Class

========= 3010a24a_2deb_43c8_9db4_cfc94c5faa5c =========
                     time  value              brick_class
0 2021-07-13 01:26:18.083 -0.107  Electrical_Power_Sensor
1 2021-07-13 01:31:17.035 -0.042  Electrical_Power_Sensor
2 2021-07-13 01:36:13.143  0.054  Electrical_Power_Sensor
3 2021-07-13 01:50:12.735 -0.016  Electrical_Power_Sensor
4 2021-07-13 01:55:12.150 -0.164  Electrical_Power_Sensor

========= 405896a1_c195_422d_a0aa_18d2eca6a61d =========
                     time  value                   brick_class
0 2021-01-01 00:03:16.153   23.5  Cooling_Temperature_Setpoint
1 2021-01-01 00:13:44.730   23.5  Cooling_Temperature_Setpoint
2 2021-01-01 00:23:16.055   23.5  Cooling_Temperature_Setpoint
3 2021-01-01 00:33:16.

## Query the building model:

Some arbitrary sparql query:

In [14]:
q = """
SELECT ?s ?p ?o WHERE {
    ?s ?p ?o
} LIMIT 5
"""

Get the results as RDFLib rows:

In [15]:
res = db.query(q)
for r in res:
    print(r)

(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#68b3266f_360c_4bbf_9eef_873275c8671a'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Return_Damper'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#e700973b_9070_4ec8_9e8d_10cdf1fd209d.9a7697b2_908b_44e2_9d47_24a3c5947254.9661bdcc_e598_421d_82f4_97a0dc5863de'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#isPointOf'), rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#e700973b_9070_4ec8_9e8d_10cdf1fd209d'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6

Get the results as a DataFrame:

In [16]:
res = db.query(q, return_df=True)
res.head()

,o,p,s
0,https://brickschema.org/schema/Brick#Return_Da...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
1,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...,https://brickschema.org/schema/Brick#isPointOf,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
2,https://brickschema.org/schema/Brick#Current_S...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
3,nfa6d86a1317343a6b9338a3afd99fa9cb87,https://brickschema.org/schema/Brick#electrica...,dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/s...
4,1,https://brickschema.org/schema/Brick#value,nfa6d86a1317343a6b9338a3afd99fa9cb450


## Query the brick schema:

Get the results as RDFLib rows:

In [17]:
res = db.query(q, graph='schema')
for r in res:
    print(r)

(rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b583'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b3673'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b20'))
(rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b3983'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b3984'))
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Parameter'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('https://brickschema.org/schema/Brick#Point'))
(rdflib.term.BNode('n7042cebd38ca462295ea4cec230891f3b3470'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Restriction'))


Get the results as a DataFrame:

In [18]:
res = db.query(q, graph='schema', return_df=True)
res.head()

,o,p,s
0,http://www.w3.org/2002/07/owl#Class,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,n7042cebd38ca462295ea4cec230891f3b583
1,n7042cebd38ca462295ea4cec230891f3b20,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,n7042cebd38ca462295ea4cec230891f3b3673
2,n7042cebd38ca462295ea4cec230891f3b3984,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,n7042cebd38ca462295ea4cec230891f3b3983
3,https://brickschema.org/schema/Brick#Point,http://www.w3.org/2000/01/rdf-schema#subClassOf,https://brickschema.org/schema/Brick#Parameter
4,http://www.w3.org/2002/07/owl#Restriction,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,n7042cebd38ca462295ea4cec230891f3b3470


## Remove the RDF prefixes from the query results:

In [19]:
res = db.query(q, return_df=True, defrag=True)
res.head()

,o,p,s
0,Return_Damper,type,68b3266f_360c_4bbf_9eef_873275c8671a
1,e700973b_9070_4ec8_9e8d_10cdf1fd209d,isPointOf,e700973b_9070_4ec8_9e8d_10cdf1fd209d.9a7697b2_...
2,Current_Sensor,type,6b498416_1218_453d_a09e_64a2e2aebe24.f246a8c4_...
3,nfa6d86a1317343a6b9338a3afd99fa9cb87,electricalPhaseCount,70d05bb1_6810_435a_bcac_8166afba0ba6.7f8a3b58_...
4,1,value,nfa6d86a1317343a6b9338a3afd99fa9cb450


## Get the building model knowledge graph itself:

In [20]:
g = db.model
i = 0
for s, p, o in g:
    print(s, p, o)
    i += 1
    if i == 5:
        break

dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#68b3266f_360c_4bbf_9eef_873275c8671a http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://brickschema.org/schema/Brick#Return_Damper
dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#e700973b_9070_4ec8_9e8d_10cdf1fd209d.9a7697b2_908b_44e2_9d47_24a3c5947254.9661bdcc_e598_421d_82f4_97a0dc5863de https://brickschema.org/schema/Brick#isPointOf dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#e700973b_9070_4ec8_9e8d_10cdf1fd209d
dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#6b498416_1218_453d_a09e_64a2e2aebe24.f246a8c4_e371_44e5_be8c_8e310eb39965 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://bricksc

## Get the brick schema knowledge graph itself:

In [21]:
g = db.schema
i = 0
for s, p, o in g:
    print(s, p, o)
    i += 1
    if i == 5:
        break

n7042cebd38ca462295ea4cec230891f3b583 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
n7042cebd38ca462295ea4cec230891f3b3673 http://www.w3.org/1999/02/22-rdf-syntax-ns#first n7042cebd38ca462295ea4cec230891f3b20
n7042cebd38ca462295ea4cec230891f3b3983 http://www.w3.org/1999/02/22-rdf-syntax-ns#rest n7042cebd38ca462295ea4cec230891f3b3984
https://brickschema.org/schema/Brick#Parameter http://www.w3.org/2000/01/rdf-schema#subClassOf https://brickschema.org/schema/Brick#Point
n7042cebd38ca462295ea4cec230891f3b3470 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Restriction


## Get the time series data itself:

In [22]:
data = db.data
i = 0
for k, v in data.items():
    print(f'{k}: {len(v)} data points')
    i += 1
    if i == 5:
        break

6f502ba0_77fb_43db_b6dc_48e68d5c822b: 51193 data points
3c9ca09b_8952_45da_a066_bcb645cbbc68: 51194 data points
d60c44c3_62fc_409a_a751_8b3ddeb9f197: 51194 data points
eb0d5916_d942_4e2d_8129_60b83d984fdd: 51200 data points
78078042_e16e_425b_9ec8_605aa149274e: 51188 data points
